In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Urumqi,43.8010,87.6005,53.60,16,0,6.71,CN,2021-04-26 04:46:07
1,1,Atuona,-9.8000,-139.0333,79.95,76,77,19.13,PF,2021-04-26 04:46:00
2,2,Vaini,-21.2000,-175.2000,82.40,89,75,4.61,TO,2021-04-26 04:46:08
3,3,Champerico,14.3000,-91.9167,66.99,95,90,3.89,GT,2021-04-26 04:46:08
4,4,Avarua,-21.2078,-159.7750,78.80,78,20,5.75,CK,2021-04-26 04:46:01


In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [10]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [20]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [32]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.count()

City_ID       157
City          157
Lat           157
Lng           157
Max Temp      157
Humidity      157
Cloudiness    157
Wind Speed    157
Country       157
Date          157
dtype: int64

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Urumqi,43.8010,87.6005,53.60,16,0,6.71,CN,2021-04-26 04:46:07
1,1,Atuona,-9.8000,-139.0333,79.95,76,77,19.13,PF,2021-04-26 04:46:00
2,2,Vaini,-21.2000,-175.2000,82.40,89,75,4.61,TO,2021-04-26 04:46:08
3,3,Champerico,14.3000,-91.9167,66.99,95,90,3.89,GT,2021-04-26 04:46:08
4,4,Avarua,-21.2078,-159.7750,78.80,78,20,5.75,CK,2021-04-26 04:46:01


In [33]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,79.95,-9.8000,-139.0333,
2,Vaini,TO,82.40,-21.2000,-175.2000,
4,Avarua,CK,78.80,-21.2078,-159.7750,
7,Yenagoa,NG,78.15,4.9247,6.2642,
14,Kavieng,PG,84.13,-2.5744,150.7967,
15,Cap Malheureux,MU,78.01,-19.9842,57.6142,
20,Kapaa,US,75.20,22.0752,-159.3190,
21,Nhulunbuy,AU,82.40,-12.2333,136.7667,
24,Rikitea,PF,77.81,-23.1203,-134.9692,
30,Cartagena,CO,80.60,10.3997,-75.5144,


In [34]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [39]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [42]:
hotel_df.head(50)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Atuona,PF,79.95,-9.8000,-139.0333,Villa Enata
2,Vaini,TO,82.40,-21.2000,-175.2000,Keleti Beach Resort
4,Avarua,CK,78.80,-21.2078,-159.7750,Paradise Inn
7,Yenagoa,NG,78.15,4.9247,6.2642,Aridolf Resort Wellness and Spa
14,Kavieng,PG,84.13,-2.5744,150.7967,Nusa Island Retreat
15,Cap Malheureux,MU,78.01,-19.9842,57.6142,Pereybere Hotel & Spa
20,Kapaa,US,75.20,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
21,Nhulunbuy,AU,82.40,-12.2333,136.7667,East Arnhem Real Estate
24,Rikitea,PF,77.81,-23.1203,-134.9692,Pension Maro'i
30,Cartagena,CO,80.60,10.3997,-75.5144,Hotel Colombia Real - Cartagena


In [49]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [47]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
